In [1]:
import os
import json
import csv
import re
import pandas as pd
import sentence_transformers
import umap.umap_ as umap
import matplotlib.pyplot as plt
import altair as alt
import langdetect 
# import multiprocessing


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Library/Frameworks/Python.framework/V

In [2]:
model = sentence_transformers.SentenceTransformer('all-mpnet-base-v2', device="cpu")

In [3]:
PFC = pd.read_csv('./PFC.csv')
PFC.head()


,conversation,Unnamed: 1,snippet,highlight,highlighter,topic,topic2,notes,story/solution/observatioin,used in report?
0,"\nMadison, WI 160",160.0,"It's like yeah, if one person is in this...thi...",https://app.lvn.org/highlight/897699,NaN,Power and Trust,Housing,"priority, police and housing",observation,yes
1,"\nMadison, WI 63",63.0,used to have all my friends talk about how gre...,https://app.lvn.org/highlight/886330,NaN,Police and Racism,NaN,"priority, little kid walking by himself gettin...",story,NaN
2,"\nMadison, WI 99",99.0,"So, it was on the news, that a teacher called ...",https://app.lvn.org/highlight/897695,NaN,Police and Youth,Power and Trust,Incident with little girl gettting pushed by p...,story,yes
3,\nNancy 2-5-19,71.0,"Well, just our everyday kids in high school wh...",https://app.lvn.org/highlights?c=1&page=1&q=bu...,NaN,Police and Youth,Police and Racism,priority,observation,yes
4,\nPFC Conversation with Just Dane,NaN,"Well, growing up, one of the first values, or ...",https://app.lvn.org/highlight/867425,NaN,Power and Trust,Police and Racism,prioriy,story,yes


In [10]:
snippet_ids = []
proportional_local = []
conversation_list = []
word_list = []
collection = []

print(len(PFC))
for i in range(0, len(PFC)):
    words = PFC["snippet"][i]
    highlight_id = str(PFC["highlight"][i]).split("/")[-1]
    conversation = PFC["conversation"][i]
    snippet_ids.append(highlight_id)
    word_list.append(words)        
    conversation_list.append(conversation)

136


In [13]:
len(snippet_ids), len(word_list), len(conversation_list)

(136, 136, 136)

In [14]:
conv_ids, texts = zip(*((snippet_ids[i], word_list[i]) for i in range(0, len(word_list))))
embeddings = model.encode(texts,show_progress_bar=True)
embeddings

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

array([[-0.00117862,  0.0703243 ,  0.02006223, ...,  0.00983872,
         0.02781419,  0.02130772],
       [ 0.01639344,  0.06621193,  0.01331168, ...,  0.03725008,
         0.04238164, -0.01484849],
       [ 0.04410992,  0.02163805,  0.01931418, ...,  0.03562592,
        -0.02445206,  0.00630008],
       ...,
       [ 0.06568735,  0.00518683,  0.00286661, ..., -0.00373769,
         0.04800443,  0.01033725],
       [ 0.06935258,  0.02793061, -0.00202731, ..., -0.02456606,
         0.05361386,  0.01753427],
       [ 0.03005826,  0.10604487,  0.00355017, ..., -0.01215767,
         0.00800674,  0.00903963]], dtype=float32)

In [15]:
len(embeddings)

136

In [17]:
reducer = umap.UMAP(n_components=2)
embedding_reduce = reducer.fit_transform(embeddings)
embedding_reduce.shape

(136, 2)

In [40]:
nodes = []
edges = []
rootxlist = []
rootylist = []
convo_list = []
convo_index_list = []
past_convo = 0
for i in range(0, len(word_list)):
    current_convo = conversation_list[i]
    current_id = str(snippet_ids[i])
    x = embedding_reduce[i][0]
    y = embedding_reduce[i][1]
    word = word_list[i]
    if "yes" in str(PFC["used in report?"][i]):
        used = True
    else:
        used = False
    if type(PFC["topic"][i]) == str: 
        topic1 = PFC["topic"][i]
    else:
        topic1 = 'none'
    if type(PFC["topic2"][i]) == str: 
        topic2 = PFC["topic2"][i]    
    else:
        topic2 = 'none'
    if "priority" in str(PFC["notes"][i]):
        priority = True
    else: 
        priority = False
    if type(PFC["notes"][i]) == str:
        notes = PFC["notes"][i]
    else:
        notes = "none"
    if type(PFC["story/solution/observatioin"][i]) == str: 
        story = PFC["story/solution/observatioin"][i]
    else:
        story = "none"

    nodes.append({
        "words": word,
        "rootx": float(translate(x, 5.8014817, 11, 0, 1200)),
        "rooty": float(translate(y, -0.2, 6, 0, 800)),
        "topic1": topic1,
        "topic2": topic2,
        "used": used,
        "story": story,
        "priority": priority,
        "notes": notes
    })

network = {"nodes": nodes}

In [26]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [41]:

with open('./PFC_embed.json', 'w') as f:
    json.dump(network, f) 